# uploading the vector embedding that we generated in the previous collab session into the qdrant database

In [ ]:
!pip install qdrant-client sentence-transformers


In [28]:
import qdrant_client
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from qdrant_client import models
from sentence_transformers import SentenceTransformer
import os
import pandas as pd
import numpy as np

In [6]:
# QDRANT_SERVER: URL of the Qdrant server
os.environ['QDRANT_SERVER'] = 'YOUR_QDRANT_URL'
# QDRANT_KEY: API key for authentication with the Qdrant server
os.environ['QDRANT_KEY'] = 'YOUR_QDRANT_API_KEY'
#create a collection name
COLLECTION_NAME="bigbasket_vectors"
# create a vector field name
vector_field_name="bigbasket_vectors_field"

#creating a qdrant_client to store collections
client = qdrant_client.QdrantClient(
        host=os.getenv("QDRANT_SERVER"),
    api_key=os.getenv("QDRANT_KEY")
    )


# Load saved embeddings & Load preprocessed bigbasket dataset

In [21]:
# Load a CSV file into a pandas DataFrame
# The file 'bigBasketProducts.csv' contains product data from BigBasket
df=pd.read_csv('/content/bigBasketProducts.csv')

# Replace missing values in the 'rating' column with 0
# This assumes that a missing rating can be treated as a 0 rating
df['rating'].fillna(0, inplace=True)

# Fill any other missing values in the DataFrame with 'NA'
# This handles missing data in other columns
df.fillna("NA", inplace=True)

# Load a NumPy array from a file
# The file 'vectors_bigbasket.npy' contains precomputed vectors, likely for the products
vectors = np.load('/content/vectors_bigbasket.npy')



In [22]:
# Convert the DataFrame into a list of dictionaries
payload = df.to_dict('records')

In [16]:
# Recreating a collection in the Qdrant client
client.recreate_collection(
            collection_name=COLLECTION_NAME,
            vectors_config={
                vector_field_name: models.VectorParams(
                    size=384,   # The size of the vector
                    distance=models.Distance.COSINE  # The type of distance metric to be used (Cosine distance)
                )
            }
        )

True

Uploading data to the specified collection in the Qdrant client

In [23]:
client.upload_collection(
            collection_name=COLLECTION_NAME,
            vectors={
                vector_field_name: vectors
            },
            payload=payload,
            ids=None,
            batch_size=256
        )

# Searching on the qdrant databse

In [76]:
class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
        self.qdrant_client = QdrantClient(os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY"))

    def search(self, text: str):
        # Convert the search text to a vector using the sentence transformer model
        vector = self.model.encode(text).tolist()

        # Perform a search in the Qdrant collection using the generated vector
        hits = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=(vector_field_name,vector),
            query_filter=None,
            limit=7
        )

        result = [hit.payload for hit in hits]
        return result

In [80]:
neural_searcher = NeuralSearcher(collection_name=COLLECTION_NAME)

# Search for a query
query = 'Strawberry Ice Cream'
results = neural_searcher.search(query)
df = pd.DataFrame(results)
df

,brand,category,description,index,market_price,product,rating,sale_price,sub_category,type
0,Haagen-Dazs,Gourmet & World Food,"Sweet, summer strawberries are introduced to f...",19639,750.0,Ice Cream - Strawberry,0.0,750.00,Dairy & Cheese,Gourmet Ice Cream
1,THE BROOKLYN CREAMERY,Gourmet & World Food,Luscious and delicious strawberry medium-fat i...,9077,325.0,Summer Strawberries Ice Cream,0.0,243.75,Dairy & Cheese,Gourmet Ice Cream
2,Haagen-Dazs,Gourmet & World Food,"Haagen-Dazs strawberry ice cream is sweet, sum...",84,225.0,Strawberry Ice Cream,0.0,225.00,Dairy & Cheese,Gourmet Ice Cream
3,Amul,"Bakery, Cakes & Dairy",Utterly delicious ice cream made from fresh mi...,8086,200.0,Real Ice Cream - Fruit 'n' Nut Fantasy,3.6,200.00,Ice Creams & Desserts,Ice Creams
4,London Dairy,Gourmet & World Food,"A 3 layered paragon, comprising a riot of flav...",11585,135.0,"Ice Cream - Berrys & Cream, Stick",0.0,135.00,Dairy & Cheese,Gourmet Ice Cream
5,Haagen-Dazs,Gourmet & World Food,"Flavours to choose from, you will be spoilt fo...",13172,750.0,Ice Cream - Blueberries & Cream,0.0,750.00,Dairy & Cheese,Gourmet Ice Cream
6,London Dairy,Gourmet & World Food,A rich & creamy strawberry ice cream crafted w...,11219,450.0,Ice Cream - Natural Strawberry,0.0,450.00,Dairy & Cheese,Gourmet Ice Cream
